In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import tensorflow_datasets as tfds
import numpy as np



## Use Cats & Dogs Dataset Load a subset of the data
##Create random bounding box in each image
## Define a model that is able to do both:
## 1. Classify the image either cat or dog
## 2. Predict the bounding box that was generated in the begging

In [ ]:
# load data subset

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:10%]', 'train[10%:15%]'],
    as_supervised=True,
    with_info=True
)

img_size = 128

#to create t random bounding box you need to pick two random x and y and it starts with minium x and then starts with min y then max x nad max y

# def preprocess(image, label):
    image = tf.image.resize(image, (img_size, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    # Random bounding box [x_min, y_min, x_max, y_max] in normalized coords
    x_min = tf.random.uniform([], 0, 0.5)
    y_min = tf.random.uniform([], 0, 0.5)
    x_max = tf.random.uniform([], 0.5, 1.0)
    y_max = tf.random.uniform([], 0.5, 1.0)
    bbox = tf.stack([x_min, y_min, x_max, y_max])
    return image, (label, bbox)

ds_train = ds_train.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)
